In [2]:
import re
import pandas as pd
import numpy as np

## Plotting Libraries
import seaborn as sns
import matplotlib.pyplot as plt

## Pytorch Imports
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data
import torch.nn.functional as F
import torch.optim as optim

## NLP Libraries
import spacy
from sklearn.model_selection import train_test_split
from nltk import download
import gensim
from nltk.corpus import stopwords
spacy_en = spacy.load('en')
download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rsilveira79/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
## Checking if GPU is available
torch.cuda.is_available()

True

In [4]:
a = torch.cuda.FloatTensor([1])
print("{} - {}".format(type(a),type(a[0])))

<class 'torch.cuda.FloatTensor'> - <class 'float'>


## 1. Reading Dataset

In [5]:
train = pd.read_csv('train.csv')
print(len(train))
train.tail()

19579


,id,text,author
19574,id17718,"I could have fancied, while I looked at it, th...",EAP
19575,id08973,The lids clenched themselves together as if in...,EAP
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
19577,id17513,"For an item of news like this, it strikes us i...",EAP
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL


In [6]:
test = pd.read_csv('test.csv')
print(len(test))
test.tail()

8392


,id,text
8387,id11749,All this is now the fitter for my purpose.
8388,id10526,I fixed myself on a wide solitude.
8389,id13477,It is easily understood that what might improv...
8390,id13761,"Be this as it may, I now began to feel the ins..."
8391,id04282,"Long winded, statistical, and drearily genealo..."


In [7]:
## Checking dataset unbalance

In [13]:
EAP = train[train['author']=='EAP'].reset_index()
EAP_size = len(EAP)
print(EAP_size)
EAP.tail()

7900


,index,id,text,author
7895,19572,id03325,But these and other difficulties attending res...,EAP
7896,19574,id17718,"I could have fancied, while I looked at it, th...",EAP
7897,19575,id08973,The lids clenched themselves together as if in...,EAP
7898,19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP
7899,19577,id17513,"For an item of news like this, it strikes us i...",EAP


In [12]:
HPL = train[train['author']=='HPL'].reset_index()
HPL_size = len(HPL)
print(HPL_size)
HPL.tail()

5635


,index,id,text,author
5630,19554,id07976,"They admitted they had been drunk, but both vo...",HPL
5631,19559,id18823,When a fumbling came in the nearer casements h...,HPL
5632,19561,id08678,Average people in society and business New Eng...,HPL
5633,19571,id14420,"My watch was still going, and told me that the...",HPL
5634,19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL


In [11]:
MWS = train[train['author']=='MWS'].reset_index()
MWS_size = len(MWS)
print(MWS_size)
MWS.tail()

6044


,index,id,text,author
6039,19563,id10563,Yet from whom has not that rude hand rent away...,MWS
6040,19566,id00832,"These reflections made our legislators pause, ...",MWS
6041,19569,id26790,Once my fancy was soothed with dreams of virtu...,MWS
6042,19570,id14263,"Nay, you may have met with another whom you ma...",MWS
6043,19573,id07567,Stress of weather drove us up the Adriatic Gul...,MWS


In [15]:
EAP[0:HPL_size].tail()

,index,id,text,author
5630,13951,id16972,I saw them now even more unequivocally than I ...,EAP
5631,13952,id05477,For some time his countenance had been losing ...,EAP
5632,13954,id17337,"""It is done it is most cheerfully agreed.",EAP
5633,13956,id05256,"""At this stage of my reflections I endeavored ...",EAP
5634,13959,id12599,Coincidences ten times as remarkable as this t...,EAP


In [25]:
train_undersampled = pd.concat([EAP[0:HPL_size],HPL,MWS[0:HPL_size]], ignore_index=True)
train_undersampled.drop(['index'],axis=1,inplace=True)
train_undersampled = train_undersampled.sample(frac=1).reset_index(drop=True)
train_undersampled.tail()

,id,text,author
16900,id22081,"Wilson, this is your property.""",EAP
16901,id05401,At length there seemed to pass a violent and s...,EAP
16902,id10807,He endeavoured to guide with prudence the stee...,MWS
16903,id18870,The readiness with which I fell into a plan of...,HPL
16904,id09898,"And where Nyarlathotep went, rest vanished; fo...",HPL


In [27]:
(len(train))

19579

## 3. Preprocessing

In [28]:
def transformText(text, do_stop=False, do_stem=False):
    
    stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    
    if (do_stop==True):
        filtered_words = [word for word in text.split() if word not in stops]
    else:
        filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=2)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    if (do_stem==True):
        # Stemming
        text = gensim.parsing.preprocessing.stem_text(text)
    return text

In [30]:
train_undersampled['phrase_preprocessed']=train_undersampled['text'].apply(lambda x: transformText(x,do_stop=False, do_stem=False))
train_undersampled.head()

,id,text,author,phrase_preprocessed
0,id21564,"Now I am the first man in the state, burthen o...",MWS,now am the first man in the state burthen of e...
1,id00662,But for one thing Old Bugs would have been an ...,HPL,but for one thing old bugs would have been an ...
2,id03160,His habits were unhinged; his restless mind ro...,MWS,his habits were unhinged his restless mind rou...
3,id15401,"These Great Old Ones, Castro continued, were n...",HPL,these great old ones castro continued were not...
4,id00866,I guess I done a bit by tellin' Selectman Mowr...,HPL,guess done bit by tellin selectman mowry what ...


In [31]:
test['phrase_preprocessed']=test['text'].apply(lambda x: transformText(x,do_stop=False, do_stem=False))
test.head()

,id,text,phrase_preprocessed
0,id02310,"Still, as I urged our leaving Ireland with suc...",still as urged our leaving ireland with such i...
1,id24541,"If a fire wanted fanning, it could readily be ...",if fire wanted fanning it could readily be fan...
2,id00134,And when they had broken down the frail door t...,and when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...,while was thinking how should possibly manage ...
4,id04081,I am not sure to what limit his knowledge may ...,am not sure to what limit his knowledge may ex...


## 4. Train/Test split, Vocab

In [32]:
x_train, x_valid, y_train, y_valid = train_test_split(train_undersampled['phrase_preprocessed'],
                                                      train_undersampled['author'], 
                                                      test_size=0.2)

In [33]:
x_test = np.array(test['phrase_preprocessed'])
x_test

array([ 'still as urged our leaving ireland with such inquietude and impatience my father thought it best to yield ',
       'if fire wanted fanning it could readily be fanned with newspaper and as the government grew weaker have no doubt that leather and iron acquired durability in proportion for in very short time there was not pair of bellows in all rotterdam that ever stood in need of stitch or required the assistance of hammer ',
       'and when they had broken down the frail door they found only this two cleanly picked human skeletons on the earthen floor and number of singular beetles crawling in the shadowy corners ',
       ...,
       'it is easily understood that what might improve closely scrutinized detail may at the same time injure general or more distantly observed effect ',
       'be this as it may now began to feel the inspiration of burning hope and at length nurtured in my secret thoughts stern and desperate resolution that would submit no longer to be enslaved ',

In [34]:
## Build Vocabulary
word_to_ix = {}
for sent in list(x_train) + list(x_valid) + list(x_test):
    for word in sent.split():
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

In [35]:
print("Vocabulary size: {}".format(len(word_to_ix)))

Vocabulary size: 27369


In [36]:
label_to_ix = { "EAP": 0, "MWS": 1, "HPL": 2 }

In [37]:
VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = len(label_to_ix)
VOCAB_SIZE, NUM_LABELS

(27369, 3)

## 5. Making dataset iterable

In [38]:
train_data=list(zip(x_train,y_train))
train_data[0:5]

[('the merriment of the hour was an unholy mockery of the sorrows of man ',
  'MWS'),
 ('but the language of the evidence speaks of the strip in question as found around the neck fitting loosely and secured with hard knot ',
  'EAP'),
 ('he could he was sure get out by midnight though it is characteristic of him that this thought was untinged with eerie implications ',
  'HPL'),
 ('in the meantime the lunatics had jolly season of it that you may swear ',
  'EAP'),
 ('klenze though not my mental equal was much better than no one ', 'HPL')]

In [39]:
valid_data=list(zip(x_valid,y_valid))
valid_data[0:5]

[('the true wretchedness indeed the ultimate woe is particular not diffuse ',
  'EAP'),
 ('if the study to which you apply yourself has tendency to weaken your affections and to destroy your taste for those simple pleasures in which no alloy can possibly mix then that study is certainly unlawful that is to say not befitting the human mind ',
  'MWS'),
 ('that glimpse like all dread glimpses of truth flashed out from an accidental piecing together of separated things in this case an old newspaper item and the notes of dead professor ',
  'HPL'),
 ('then as remained paralysed with fear he found his voice and in his dying breath screamed forth those words which have ever afterward haunted my days and my nights ',
  'HPL'),
 ('observed however that the shutters of the fourth story were of the peculiar kind called by parisian carpenters ferrades kind rarely employed at the present day but frequently seen upon very old mansions at lyons and bordeaux ',
  'EAP')]

In [40]:
def make_context_vector(seq, to_ix):
    idxs = [to_ix[w] for w in seq.split()]
    tensor = torch.cuda.LongTensor(idxs)
    return tensor

In [41]:
def make_target(label, label_to_idx):
    return torch.cuda.LongTensor([label_to_idx[label]])

## 6. Model - LSTM Classifier with Word2vec

In [42]:
!ls ../../vectors/

glove.42B.300d.txt		       wget-log.1
GoogleNews-vectors-negative300.bin.gz  wiki-news-300d-1M-subword.vec
wget-log


In [43]:
glove_path = '../../vectors/glove.42B.300d.txt'

In [44]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [45]:
glove_vector = loadGloveModel(glove_path)

Loading Glove Model
Done. 1917494  words loaded!


In [46]:
glove_vector['start'][0:100]

array([-0.3013  ,  0.49315 , -0.47456 , -0.55259 , -0.079275, -0.069589,
       -3.5792  ,  0.28671 , -0.034099, -0.23643 , -0.042276,  0.25939 ,
       -0.080374, -0.25137 ,  0.22347 ,  0.22123 ,  0.077188,  0.41157 ,
       -0.1154  , -0.065081, -0.1549  , -0.08466 ,  0.078873, -0.109   ,
       -0.018008,  0.2591  ,  0.09544 , -0.33333 , -0.24942 , -0.29616 ,
       -0.60454 , -0.11145 ,  0.21661 , -0.17045 ,  0.2329  ,  0.24699 ,
       -0.37996 , -0.035888, -0.035235,  0.14275 , -0.13491 ,  0.018954,
        0.35121 , -0.34182 ,  0.041529, -0.48883 ,  0.29649 ,  0.25711 ,
        0.21757 , -0.04452 ,  0.49266 ,  0.30706 , -0.30375 ,  0.12427 ,
       -0.090547,  0.055427, -0.357   ,  0.049935, -0.12842 ,  0.037939,
        0.20706 ,  0.24645 ,  0.45067 ,  0.26154 , -0.39262 ,  0.10625 ,
       -0.090539,  0.13802 ,  0.088777, -0.4219  , -0.14283 ,  0.053959,
       -0.40817 , -0.13262 , -0.44289 ,  0.046249,  0.53221 ,  0.030767,
        0.06197 , -0.18862 , -0.28016 ,  0.09877 , 

In [47]:
#from gensim.models import KeyedVectors

In [48]:
#w2v = KeyedVectors.load_word2vec_format('../../vectors/GoogleNews-vectors-negative300.bin', binary = True)

In [49]:
W2V_DIM = 300
## standard deviation to use
sd = 1/np.sqrt(W2V_DIM)
## Random initialization
weights = np.random.normal(0, scale=sd, size=[VOCAB_SIZE, W2V_DIM])
weights = weights.astype(np.float32)

In [50]:
for word in word_to_ix:
    id = word_to_ix.get(word,None)
    if id is not None:
        try:
            #weights[id]=w2v.wv.word_vec(word)
            weights[id]=glove_vector[word]
        except:
            weights[id]=np.random.normal(0, scale=sd, size=[1, W2V_DIM]) ## If word not present, initialize randomly

In [51]:
word_to_ix

{'the': 0,
 'merriment': 1,
 'of': 2,
 'hour': 3,
 'was': 4,
 'an': 5,
 'unholy': 6,
 'mockery': 7,
 'sorrows': 8,
 'man': 9,
 'but': 10,
 'language': 11,
 'evidence': 12,
 'speaks': 13,
 'strip': 14,
 'in': 15,
 'question': 16,
 'as': 17,
 'found': 18,
 'around': 19,
 'neck': 20,
 'fitting': 21,
 'loosely': 22,
 'and': 23,
 'secured': 24,
 'with': 25,
 'hard': 26,
 'knot': 27,
 'he': 28,
 'could': 29,
 'sure': 30,
 'get': 31,
 'out': 32,
 'by': 33,
 'midnight': 34,
 'though': 35,
 'it': 36,
 'is': 37,
 'characteristic': 38,
 'him': 39,
 'that': 40,
 'this': 41,
 'thought': 42,
 'untinged': 43,
 'eerie': 44,
 'implications': 45,
 'meantime': 46,
 'lunatics': 47,
 'had': 48,
 'jolly': 49,
 'season': 50,
 'you': 51,
 'may': 52,
 'swear': 53,
 'klenze': 54,
 'not': 55,
 'my': 56,
 'mental': 57,
 'equal': 58,
 'much': 59,
 'better': 60,
 'than': 61,
 'no': 62,
 'one': 63,
 'new': 64,
 'system': 65,
 'your': 66,
 'own': 67,
 'invention': 68,
 'sunny': 69,
 'weather': 70,
 'glad': 71,
 'bar'

In [52]:
glove_vector["confessed"][0:50]

array([ -4.27130000e-01,   1.34660000e-01,  -3.49210000e-01,
         4.81920000e-01,  -4.93770000e-01,  -3.87940000e-02,
        -1.36500000e+00,  -6.52720000e-02,  -2.83830000e-01,
        -5.63370000e-01,  -2.60910000e-01,  -2.14390000e-02,
        -1.68270000e-01,  -1.90030000e-01,   1.46610000e-01,
         4.86780000e-01,   1.23060000e-01,  -3.40320000e-01,
         1.81250000e-01,   5.68720000e-01,  -1.14460000e-03,
        -5.96980000e-01,   1.30130000e-01,   7.11450000e-02,
        -5.22910000e-02,   1.61650000e-01,  -2.21380000e-01,
        -4.95220000e-01,   1.44850000e-01,  -1.23160000e-01,
         9.43030000e-02,   2.70700000e-01,   3.47260000e-02,
         4.33490000e-01,   8.71990000e-02,  -1.09690000e-01,
        -3.34760000e-01,   2.29290000e-01,   1.56970000e-01,
         1.58630000e-01,  -1.97850000e-01,   5.14440000e-02,
        -4.40730000e-01,   2.08360000e-01,   7.55970000e-02,
        -3.68940000e-01,  -1.75310000e-02,   5.62600000e-02,
         3.93250000e-01,

In [53]:
idx=word_to_ix['confessed']

In [54]:
weights[idx][0:50]

array([ -4.27130014e-01,   1.34660006e-01,  -3.49209994e-01,
         4.81920004e-01,  -4.93770003e-01,  -3.87939997e-02,
        -1.36500001e+00,  -6.52720034e-02,  -2.83829987e-01,
        -5.63369989e-01,  -2.60910004e-01,  -2.14389991e-02,
        -1.68270007e-01,  -1.90029994e-01,   1.46610007e-01,
         4.86779988e-01,   1.23060003e-01,  -3.40319991e-01,
         1.81250006e-01,   5.68719983e-01,  -1.14459998e-03,
        -5.96979976e-01,   1.30129993e-01,   7.11449981e-02,
        -5.22909984e-02,   1.61650002e-01,  -2.21379995e-01,
        -4.95220006e-01,   1.44850001e-01,  -1.23159997e-01,
         9.43029970e-02,   2.70700008e-01,   3.47260013e-02,
         4.33490008e-01,   8.71990025e-02,  -1.09690003e-01,
        -3.34760010e-01,   2.29289994e-01,   1.56969994e-01,
         1.58629999e-01,  -1.97850004e-01,   5.14440015e-02,
        -4.40730006e-01,   2.08360001e-01,   7.55970031e-02,
        -3.68939996e-01,  -1.75310001e-02,   5.62600009e-02,
         3.93249989e-01,

In [76]:
W2V_DIM = 300
HIDDEN_DIM = 100
NUM_LAYERS = 3
DROPOUT = 0.7

In [77]:
class GruClassifierW2vec(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, num_layers, vocab_size, label_size, pre_trained_weights, dropout):
        super(GruClassifierW2vec, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.word_embeddings.weight.data=torch.Tensor(pre_trained_weights)
        self.gru = nn.GRU(input_size = embedding_dim,
                            hidden_size = hidden_dim,
                            num_layers = num_layers,
                            dropout = dropout)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # the first is the hidden h
        return (Variable(torch.zeros(self.num_layers, 1, self.hidden_dim)).cuda())

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        x = embeds.view(len(sentence), 1, -1)
        for i in range(self.num_layers):
            gru_out, self.hidden = self.gru(x, self.hidden)
        y  = self.hidden2label(gru_out[-1])
        log_probs = F.log_softmax(y)
        return log_probs

In [78]:
model = GruClassifierW2vec(embedding_dim=W2V_DIM,
                            hidden_dim=HIDDEN_DIM,
                            num_layers=NUM_LAYERS,
                            vocab_size=VOCAB_SIZE,
                            label_size=NUM_LABELS,
                            pre_trained_weights = weights,
                            dropout = DROPOUT)

In [79]:
model.cuda()

GruClassifierW2vec (
  (word_embeddings): Embedding(27369, 300)
  (gru): GRU(300, 100, num_layers=3, dropout=0.7)
  (hidden2label): Linear (100 -> 3)
)

In [80]:
label_to_ix

{'EAP': 0, 'HPL': 2, 'MWS': 1}

In [81]:
EAP_size

7900

In [82]:
HPL_size

5635

In [83]:
MWS_size

6044

In [84]:
avg=int((EAP_size+HPL_size+MWS_size)/3)
avg

6526

In [85]:
print(float(avg/EAP_size))
print(float(avg/HPL_size))
print(float(avg/MWS_size))

0.8260759493670886
1.1581188997338066
1.0797485109199205


In [86]:
EAP_size*0.826

6525.4

In [87]:
HPL_size*1.158

6525.33

In [88]:
MWS_size*1.079

6521.476

In [89]:
label_to_ix

{'EAP': 0, 'HPL': 2, 'MWS': 1}

In [90]:
## Loss function with mask to compensate class inbalance
mask=torch.FloatTensor((0.826,1.158,1.079))
#loss_function = nn.CrossEntropyLoss(weight=mask.view(1,-1))

In [91]:
loss_function = nn.CrossEntropyLoss()
#print(loss_function.weight)
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(),lr = learning_rate)

In [92]:
sample=train_data[2][0]
sample

'he could he was sure get out by midnight though it is characteristic of him that this thought was untinged with eerie implications '

In [93]:
sample_context=Variable(make_context_vector(sample,word_to_ix)).cuda()
sample_context

Variable containing:
 28
 29
 28
  4
 30
 31
 32
 33
 34
 35
 36
 37
 38
  2
 39
 40
 41
 42
  4
 43
 25
 44
 45
[torch.cuda.LongTensor of size 23 (GPU 0)]

In [94]:
out=model(sample_context)
out

Variable containing:
-1.0951 -1.2024 -1.0078
[torch.cuda.FloatTensor of size 1x3 (GPU 0)]

In [95]:
batch_size = 10
n_iters = 2000
num_epochs = n_iters/(len(x_train) / batch_size)
num_epochs = int(num_epochs)
num_epochs

1

In [96]:
iter = 0
for epoch in range(num_epochs):
    for (sent,label) in train_data:
        # Step 1 - clear the gradients
        model.zero_grad()
        optimizer.zero_grad()
        model.hidden = model.init_hidden()
    
        ## Avoid breaking for empty input
        try:
            ## Step 2- Prepare input and label
            context_vec = Variable(make_context_vector(sent, word_to_ix)).cuda()
            target = Variable(make_target(label, label_to_ix)).cuda()
            # Step 3 - Run forward pass
            output = model(context_vec)  
            # Step 4 - Compute loss, gradients, update parameters
            loss = loss_function(output, target)
            loss.backward()
            optimizer.step()
        except:
            pass
        iter+=1      
        ## Calculate final accuracy
        if iter % 500 ==0:
            correct = 0
            total = 0
            for (sent,label) in valid_data:
                context_vec = Variable(make_context_vector(sent, word_to_ix)).cuda()
                target = Variable(make_target(label, label_to_ix)).cuda()
                output = model(context_vec)
                _,predicted = torch.max(output.data,1)
                total += target.size(0)
                correct += (predicted[0] == make_target(label, label_to_ix)).sum()
            accuracy = 100 * correct/total
            print('Iterations: {}. Loss: {}'.format(iter,loss.data[0]))

Iterations: 500. Loss: 1.0518038272857666
Iterations: 1000. Loss: 0.9179424047470093
Iterations: 1500. Loss: 1.1740186214447021
Iterations: 2000. Loss: 0.4193210005760193
Iterations: 2500. Loss: 0.9742138981819153
Iterations: 3000. Loss: 0.9318205118179321
Iterations: 3500. Loss: 0.4751545190811157
Iterations: 4000. Loss: 0.18728940188884735
Iterations: 4500. Loss: 1.9101083278656006
Iterations: 5000. Loss: 0.9721226096153259
Iterations: 5500. Loss: 0.9946901202201843
Iterations: 6000. Loss: 0.5614168643951416
Iterations: 6500. Loss: 2.71254825592041
Iterations: 7000. Loss: 2.937105178833008
Iterations: 7500. Loss: 2.4545650482177734
Iterations: 8000. Loss: 1.4660981893539429
Iterations: 8500. Loss: 1.6912579536437988
Iterations: 9000. Loss: 0.21140481531620026
Iterations: 9500. Loss: 0.3685607612133026
Iterations: 10000. Loss: 0.9854797124862671
Iterations: 10500. Loss: 0.5488322973251343
Iterations: 11000. Loss: 2.1286096572875977
Iterations: 11500. Loss: 2.3581249713897705
Iteration

In [ ]:
## Making predictions on Test set

In [ ]:
n=2
bow_vec = Variable(make_context_vector(valid_data[n][0], word_to_ix))
print("-"*20 + " INPUT "+"-"*30)
print("TRUE LABEL = {}".format(valid_data[n][1]))
print("SENTENCE = {}".format(valid_data[n][0]))
print("-"*20 + " PREDICTION "+"-"*30)
log_probs = model(bow_vec)
_,predicted = torch.max(log_probs.data,1)
print("PRED = {}".format(predicted[0]))
print("PRED = {}".format(list(label_to_ix.keys())[list(label_to_ix.values()).index(predicted[0])]))
##print("LOG_PROB = {}".format(log_probs))
print("PROBS = {}".format(F.softmax(log_probs)))

In [ ]:
from sklearn.metrics import log_loss

In [ ]:
y_valid

In [ ]:
def make_preds(model,test):
    my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
    my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
    for i in range(len(test['phrase_preprocessed'])):
        sample=test['phrase_preprocessed'][i]
        #print(sample)
        sample_context=Variable(make_context_vector(sample,word_to_ix)).cuda()
        log_prob=model(sample_context)
        probs=F.softmax(log_prob)
        my_sub.loc[i] = [test['id'][i], probs.data[0][0],probs.data[0][1],probs.data[0][2]]
    return my_sub

In [ ]:
preds=make_preds(model,test)

In [ ]:
preds.tail()

In [ ]:
preds.to_csv('roberto_new_11.csv',index=False)